In [16]:
# Import necessary PySpark SQL functions
from pyspark.sql import SparkSession
from pyspark.sql.functions import avg

# Initialize SparkSession
spark = SparkSession.builder \
    .appName("Home Sales Analysis") \
    .getOrCreate()

# Read the home_sales_revised.csv data into a Spark DataFrame
df = spark.read.csv("home_sales_revised.csv", header=True, inferSchema=True)

# Create a temporary table called home_sales
df.createOrReplaceTempView("home_sales")

# 1. What is the average price for a four-bedroom house sold for each year?
query1 = """
SELECT date_built, ROUND(AVG(price), 2) AS avg_price
FROM home_sales
WHERE bedrooms = 4
GROUP BY date_built
ORDER BY date_built
"""
result1 = spark.sql(query1)
result1.show()

# 2. What is the average price of a home for each year the home was built, 
#    that has three bedrooms and three bathrooms?
query2 = """
SELECT date_built, ROUND(AVG(price), 2) AS avg_price
FROM home_sales
WHERE bedrooms = 3 AND bathrooms = 3
GROUP BY date_built
ORDER BY date_built
"""
result2 = spark.sql(query2)
result2.show()

# 3. What is the average price of a home for each year the home was built, 
#    that has three bedrooms, three bathrooms, two floors, and is 
#    greater than or equal to 2,000 square feet?
query3 = """
SELECT date_built, ROUND(AVG(price), 2) AS avg_price
FROM home_sales
WHERE bedrooms = 3 AND bathrooms = 3 AND floors = 2 AND sqft_living >= 2000
GROUP BY date_built
ORDER BY date_built
"""
result3 = spark.sql(query3)
result3.show()

# 4. What is the average price of a home per "view" rating having 
#    an average home price greater than or equal to $350,000?
query4 = """
SELECT view, ROUND(AVG(price), 2) AS avg_price
FROM home_sales
GROUP BY view
HAVING AVG(price) >= 350000
"""
result4 = spark.sql(query4)
result4.show()

# Cache your temporary table home_sales
spark.catalog.cacheTable("home_sales")

# Check if your temporary table is cached
print("Is home_sales cached?", spark.catalog.isCached("home_sales"))

# Run the last query with cached data
result5 = spark.sql(query4)
result5.show()

# Partition by the "date_built" field on the formatted parquet home sales data
df.write.partitionBy("date_built").mode("overwrite").parquet("home_sales_partitioned.parquet")


# Create a temporary table for the parquet data
partitioned_df = spark.read.parquet("home_sales_partitioned.parquet")
partitioned_df.createOrReplaceTempView("home_sales_partitioned")

# Run the last query with parquet data
result6 = spark.sql(query4)
result6.show()

# Uncache the home_sales temporary table
spark.catalog.uncacheTable("home_sales")

# Verify that the home_sales temporary table is uncached
print("Is home_sales cached?", spark.catalog.isCached("home_sales"))

# Stop SparkSession
spark.stop()


+----------+---------+
|date_built|avg_price|
+----------+---------+
|      2010|296800.75|
|      2011| 302141.9|
|      2012|298233.42|
|      2013|299999.39|
|      2014|299073.89|
|      2015|307908.86|
|      2016|296050.24|
|      2017|296576.69|
+----------+---------+

+----------+---------+
|date_built|avg_price|
+----------+---------+
|      2010|292859.62|
|      2011|291117.47|
|      2012|293683.19|
|      2013|295962.27|
|      2014|290852.27|
|      2015| 288770.3|
|      2016|290555.07|
|      2017|292676.79|
+----------+---------+

+----------+---------+
|date_built|avg_price|
+----------+---------+
|      2010|285010.22|
|      2011|276553.81|
|      2012|307539.97|
|      2013|303676.79|
|      2014|298264.72|
|      2015|297609.97|
|      2016| 293965.1|
|      2017|280317.58|
+----------+---------+

+----+----------+
|view| avg_price|
+----+----------+
|  85|1056336.74|
|  65| 736679.93|
|  53|  755214.8|
|  78|1080649.37|
|  81|1053472.79|
|  76|1058802.78|
|  91|1

Py4JJavaError: An error occurred while calling o405.parquet.
: java.lang.RuntimeException: java.io.FileNotFoundException: java.io.FileNotFoundException: HADOOP_HOME and hadoop.home.dir are unset. -see https://wiki.apache.org/hadoop/WindowsProblems
	at org.apache.hadoop.util.Shell.getWinUtilsPath(Shell.java:735)
	at org.apache.hadoop.util.Shell.getSetPermissionCommand(Shell.java:270)
	at org.apache.hadoop.util.Shell.getSetPermissionCommand(Shell.java:286)
	at org.apache.hadoop.fs.RawLocalFileSystem.setPermission(RawLocalFileSystem.java:978)
	at org.apache.hadoop.fs.RawLocalFileSystem.mkOneDirWithMode(RawLocalFileSystem.java:660)
	at org.apache.hadoop.fs.RawLocalFileSystem.mkdirsWithOptionalPermission(RawLocalFileSystem.java:700)
	at org.apache.hadoop.fs.RawLocalFileSystem.mkdirs(RawLocalFileSystem.java:672)
	at org.apache.hadoop.fs.RawLocalFileSystem.mkdirsWithOptionalPermission(RawLocalFileSystem.java:699)
	at org.apache.hadoop.fs.RawLocalFileSystem.mkdirs(RawLocalFileSystem.j